<a href="https://colab.research.google.com/github/NajmehNyr/Social-Status-/blob/main/social.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd; pd.set_option('future.no_silent_downcasting', True)

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import plotly.express as px

In [ ]:
url = 'https://docs.google.com/spreadsheets/d/1HiCx_IS2t0PLg1Blmw5XmmFyXS6oQl3H/export?format=csv'
df = pd.read_csv(url)

In [ ]:
while True:
    PerID = input("Enter the ID of the person you want to see (1-31): ")
    try:
        PerID = int(PerID)
        if 1 <= PerID <= 31:
            dfRow = df[df['ID'] == PerID]
            ndfRow = ndf[ndf['ID'] == PerID]
            break
        else: print('IDs are integers from 1 to 31. \nPlease try again.')
    except ValueError: print('Enter an integer ID between 1 to 31. \nPlease try again.')

Enter the ID of the person you want to see (1-31): 5


In [ ]:
df.head()

,ID,GENDER,MARR,RELIBIL,FAMINC,PERSINC,RESID,FATEDU,UNNES,ENTER,...,SOCMED,FRI,science,politics,culture,religon,others,nothing,FAMCOM,EDUCOM
0,1,1,1,2,30,0,3,3,۵۰ الی ۷۰ درصد,۱ الی ۳ میلیون تومان,...,۱ تا ۳ ساعت,متوسط,علمی,NaN,فرهنگی,NaN,NaN,NaN,1,3
1,2,1,1,3,60,0,4,2,۳۰ الی ۵۰ درصد,۱ الی ۳ میلیون تومان,...,۱ تا ۳ ساعت,متوسط,NaN,NaN,فرهنگی,NaN,NaN,NaN,2,4
2,3,1,1,3,15,0,2,1,۳۰ الی ۵۰ درصد,زیر ۱ میلیون تومان,...,بالای ۵ ساعت,زیاد,NaN,NaN,NaN,NaN,NaN,هیچکدام,1,2
3,4,1,1,2,17.5,0,3,1,۷۰ الی ۱۰۰ درصد,زیر ۱ میلیون تومان,...,۱ تا ۳ ساعت,متوسط,علمی,NaN,NaN,NaN,NaN,NaN,2,3
4,5,1,1,3,14,0,3,3,۵۰ الی ۷۰ درصد,زیر ۱ میلیون تومان,...,۱ تا ۳ ساعت,زیاد,NaN,NaN,NaN,NaN,سایر,NaN,1,2


**The Dataset:** This analysis belongs to Social Status dataset of psychology class of 2028 of University of X. We gathered the data in our first semester so there were many mistakes in the designed survey in that time I was the Data Analysist of this team and I alnalyzied the data in Excel.

**The Varibles**: Click here.

##Preprocessing

**Correcting Data Coleoctor's Mistakes**

Here, I will correct some of the data collector's mistakes, so having a bigger number on the scale means better performance in that task. At the same time, I will also convert my strings to numerical values, especially since they are Persian.

In [ ]:
# Replace values with new ones
# in 2 first ones: sort unique values then give them ordinal numbrs that increas by 1
REPs = {
    'UNNES':  {ValUnn: IndUnn + 1 for IndUnn, ValUnn in enumerate(np.sort(df['UNNES'].unique()))},
    'ENTER':  {ValEnt: IndEnt + 1 for IndEnt, ValEnt in enumerate(np.sort(df['ENTER'].unique()))},
    'READ':   {'زیر یکساعت': 1, 'یک تا سه ساعت': 2, 'سه تا هفت': 3, 'بالای هفت ساعت...': 4},
    'SPORT':  {'خیر ، فاقد مهارت ورزشی': 1, 'در حد مبتدی': 2, 'در حد متوسط': 3, 'بله ، در حد عالی': 4},
    'ENG':    {'فاقد آشنایی': 1, 'در حد کم': 2, 'متوسط': 3, 'زیاد': 4, 'عالی': 5},
    'VOUL':   {'خیلی ضعیف': 1, 'ضعیف': 2, 'متوسط': 3, 'خوب': 4},
    'SOCMED': {'زیر ۱ ساعت': 1, '۱ تا ۳ ساعت': 2, '۳ تا ۵ ساعت': 3, 'بالای ۵ ساعت': 4},
    'FRI':    {'خیلی کم': 1, 'کم': 2, 'متوسط': 3, 'زیاد': 4},
    'FAMCOM': {1: 3, 3: 1},
    'RESID':  {1: 5, 2: 4, 4: 2, 5: 1}}
df = df.replace(REPs)

‌

**Combining and Dropping Some of The Columns**

There are six columns related to the type of NGOs individuals are involved in.

`NGO_Types = ['science', 'culture', 'religion', 'politics', 'others', 'nothing']`.
I will count involvement in each of these columns, except for `NGO_Types['nothing']`, and then create a column named "NGOs" for them.

The nothing column isn't needed here, so I will drop it alongside marriage status since none of the individuals is married in our survey.

In [ ]:
# Envolvment in NGOs count
NGOs = ['science', 'culture', 'religon', 'politics', 'others']
for NGO in NGOs: #fill NaNs with 0 and none NaNs with 1
  df[NGO] = df[NGO].fillna(0).apply(lambda x: 1 if x != 0 else 0)
df['NGONUM'] = df[NGOs].sum(axis=1)

# Drop cloumns that are useless or not needed anymore
df.drop(['nothing', 'MARR'] + NGOs, axis=1, inplace=True)

‌

**Handeling Missing Data**

Here after I turned all my data points to numeric values I search for missing data points and fill those with appropriate values.

Columns with missing data:
*   Family Income: Median due that varibility.
*   Person's Income: 0 beacuse most of the indivisouls didn't make any money so if they didn't answer this they probebly were the same.


In [ ]:
# Convert columns to numeric, coercing errors to NaN
df = df.apply(pd.to_numeric, errors='coerce')

# Fill missing values
df['FAMINC'] = df['FAMINC'].fillna(df['FAMINC'].median())
df.at[30, "PERSINC"] = 0
df['COMP'] = df['COMP'].fillna(df['COMP'].mean()).astype(int)

In [ ]:
# Check if there're any null values left
NullLen = df.isnull().sum()
NullCol = NullLen[NullLen > 0]
if not NullCol.empty: print(f"Columns with remaining null values: {NullCol}")

‌

**Convering All the Values to Scores**

**Income varibles**: We ask pepole how much they make per month (million Rial) but those valuses need to be transfered to scores.

**Personal Income**: Only a minority had thier own outcome we have 3 scores for Personal income and 5 for Family Income.

In [ ]:
# specifying the cuts, for FAMINC create 5 cuts and 3 fpr PERSINC
FamincCuts = [np.min(df['FAMINC']) + faminc_i * (np.ptp(df['FAMINC']) / 5) for faminc_i in range(6)]
FamincCuts[-1] = np.max(df['FAMINC']) + 1 #just for fixing a minor error
PersincCuts = [0, 5, 10, 51]

# Labeling and cuts the values
df['FAMINC_SCORE'] = pd.cut(df['FAMINC'], bins=FamincCuts, labels=[1, 2, 3, 4, 5], right=False, include_lowest=True)
df['PERSINC_SCORE'] = pd.cut(df['PERSINC'], bins=PersincCuts, labels=[1, 2, 3], right=False, include_lowest=True)

‌

**Creating Our Main Varibles**

In this project we aim to mesure the classes economical, social and cultural score to compare each persons socoeconomical score to others. for that, we need those to be our main varible. and in one varible we combine social and cultural score. we also add an overall score for the overall analysis.

In [ ]:
# Create main vaibles dict and get teir mean
vars= {
    'ECO': {'FAMINC_SCORE', 'PERSINC_SCORE', 'RESID' , 'UNNES' , 'ENTER' ,  'JOBEXP'       },
    'CUL': {'GENDER'      , 'RELIBIL'      , 'FATEDU', 'READ'  , 'SPORT' ,  'COMP'  , 'ENG'},
    'SOC': {'VOUL'        , 'SOCMED'       , 'FRI'   , 'NGONUM', 'FAMCOM',  'EDUCOM'       }}
for Var, SubVar in vars.items():
  df[Var] = df[list(SubVar)].mean(axis=1)

In [ ]:
# Create Additioal varibles
df['SOCCUL'] = df[['CUL', 'SOC']].mean(axis=1)
df['OVERALL'] = df[['ECO', 'CUL', 'SOC']].mean(axis=1)
ADDITIONAL = ['SOCCUL', 'OVERALL']

# Rearrange the data[ID and INCS + vars[subvars] + list(vars) + ADDITIONAL]
NewOrder = ['ID', 'PERSINC', 'FAMINC'] + [subvar for var in vars.keys() for subvar in list(vars[var])] + list(vars.keys()) + ADDITIONAL
df = df[NewOrder]

‌

**Caculating min and max**

The theorical min and max are diffrent with the ones particioants actually choosee, so I caculated the reah min and max for normalization.

In [ ]:
# Write min and max based on survey options
MinMax = {'GENDER':[1,2], 'RELIBIL':[1,5], 'SPORT':[1,4]      , 'RESID':[1,5]  ,
          'FATEDU':[1,5],'UNNES':[1,4]   , 'ENTER':[1,4]       , 'JOBEXP':[1,3]      , 'READ':[1,4]   ,
          'COMP':[1,4]  , 'ENG':[1,5]    , 'VOUL':[1,5]        ,'SOCMED':[1,4]       , 'FRI':[1,5]    ,
          'EDUCOM':[1,5], 'NGONUM':[0,5] , 'FAMINC_SCORE':[1,5],'PERSINC_SCORE':[1,3],  'FAMCOM':[1,3],
          'FAMINC':[df['FAMINC'].min(), df['FAMINC'].max()],
          'PERSINC':[df['PERSINC'].min(), df['PERSINC'].max()]}

In [ ]:
#TODO1: Calculate min and max based on minmax dict

# For Vars
VarMin = [MinMax[SubVar][0] for SubVar in SubVars]
VarMax = [MinMax[SubVar][1] for SubVar in SubVars]
MinMaxVars = {Var: (np.mean(VarMin), np.mean(VarMax))
              for Var, SubVar in vars.items()}

# For ADDITIONALs
SOCCULMin = [MinMaxVars['SOC'][0], MinMaxVars['CUL'][0]]
SOCCULMax = [MinMaxVars['SOC'][1], MinMaxVars['CUL'][1]]

OVERALLMin = [MinMaxVars['ECO'][0], MinMaxVars['CUL'][0], MinMaxVars['SOC'][0]]
OVERALLMax = [MinMaxVars['ECO'][1], MinMaxVars['CUL'][1], MinMaxVars['SOC'][1]]

MinMaxAdd = {'SOCCUL': (np.mean(SOCCULMin), np.mean(SOCCULMax)),
             'OVERALL': (np.mean(OVERALLMin), np.mean(OVERALLMax))}


#TODO2: Updating the MInMAx dictionary with var means and additional means
MinMax.update(MinMaxVars)
MinMax.update(MinMaxAdd)
df.head()

,ID,PERSINC,FAMINC,PERSINC_SCORE,RESID,ENTER,UNNES,FAMINC_SCORE,JOBEXP,READ,...,FAMCOM,EDUCOM,SOCMED,NGONUM,VOUL,ECO,CUL,SOC,SOCCUL,OVERALL
0,1,0.0,30.0,1,3,3,3,2,1,2,...,3,3,2,2,3,2.166667,2.285714,2.666667,2.476190,2.373016
1,2,0.0,60.0,1,2,3,2,3,1,1,...,2,4,2,1,4,2.000000,1.857143,2.666667,2.261905,2.174603
2,3,0.0,15.0,1,4,2,2,1,3,1,...,3,2,4,0,2,2.166667,1.571429,2.500000,2.035714,2.079365
3,4,0.0,17.5,1,3,2,4,1,1,4,...,2,3,2,1,3,2.000000,2.571429,2.333333,2.452381,2.301587
4,5,0.0,14.0,1,3,2,3,1,1,1,...,3,2,2,1,3,1.833333,2.000000,2.500000,2.250000,2.111111


###Normalization

**Min-Max scaling**

I use Min-Max scaling becuse as I said theres diffrence between theorical and practical min and max which itself is important scince on one in our sample get more that 0.6 score in economics and probebly reflects the high inflation rate and broken economics of Iran.

In addition, Varibles didn't used the same scaling. we had 1-2, 1-3, 1-4, 1-5 and seems resonable to use min max scaling which will also make data interpertation more undrestandble.

In [ ]:
# Define the min_max scaling fuction

def MMScaling(InpCol):

  Min = MinMax[InpCol.name][0]
  Max = MinMax[InpCol.name][1]
  MMScaled = (InpCol - Min) / (Max - Min)
  return MMScaled

In [ ]:
# Creating the Normalized Data Frame(ndf)
ndf = df.copy()

# Identify categories and columns to normalize
NCols = ['ECO', 'CUL', 'SOC', 'SOCCUL', 'OVERALL']
for var in vars.keys():
    NCols.extend(list(vars[var]))

# Convert columns to numeric and normalizing them
for col in NCols:
    ndf[col] = pd.to_numeric(ndf[col], errors='coerce')
    ndf[col] = MMScaling(ndf[col])

In [ ]:
# Reordering the Normalized Data Frame
NewOrder2 = ['ID', 'PERSINC', 'FAMINC'] + [subvar for var in vars.keys() for subvar in list(vars[var]) if subvar not in ['FAMINC_SCORE', 'PERSINC_SCORE']] + list(vars.keys()) + ADDITIONAL
ndf = ndf[NewOrder2].round(4)
ndf.head()

,ID,PERSINC,FAMINC,RESID,ENTER,UNNES,JOBEXP,READ,FATEDU,SPORT,...,FAMCOM,EDUCOM,SOCMED,NGONUM,VOUL,ECO,CUL,SOC,SOCCUL,OVERALL
0,1,0.0,30.0,0.50,0.6667,0.6667,0.0,0.3333,0.50,0.6667,...,1.0,0.50,0.3333,0.4,0.50,0.3636,0.3961,0.5000,0.4481,0.4199
1,2,0.0,60.0,0.25,0.6667,0.3333,0.0,0.0000,0.25,0.3333,...,0.5,0.75,0.3333,0.2,0.75,0.3182,0.2792,0.5000,0.3896,0.3658
2,3,0.0,15.0,0.75,0.3333,0.3333,1.0,0.0000,0.00,0.3333,...,1.0,0.25,1.0000,0.0,0.25,0.3636,0.2013,0.4545,0.3279,0.3398
3,4,0.0,17.5,0.50,0.3333,1.0000,0.0,1.0000,0.00,0.3333,...,0.5,0.50,0.3333,0.2,0.50,0.3182,0.4740,0.4091,0.4416,0.4004
4,5,0.0,14.0,0.50,0.3333,0.6667,0.0,0.0000,0.50,0.3333,...,1.0,0.25,0.3333,0.2,0.50,0.2727,0.3182,0.4545,0.3864,0.3485


##Analysis

*analysis of overall corrlation*

In [ ]:
#TODO1: try diffrent matrixes
#TODO2: visulize the corrlations

##Seeing individual analysis

### Coresponding rows to ID

In [ ]:
display(dfRow)
display(ndfRow)

,ID,GENDER,MARR,RELIBIL,FAMINC,PERSINC,RESID,FATEDU,UNNES,ENTER,...,SOCMED,FRI,science,politics,culture,religon,others,nothing,FAMCOM,EDUCOM
1,2,1,1,3,60,0,4,2,۳۰ الی ۵۰ درصد,۱ الی ۳ میلیون تومان,...,۱ تا ۳ ساعت,متوسط,NaN,NaN,فرهنگی,NaN,NaN,NaN,2,4


,ID,PERSINC,FAMINC,RESID,ENTER,UNNES,JOBEXP,READ,FATEDU,SPORT,...,FAMCOM,EDUCOM,SOCMED,NGONUM,VOUL,ECO,CUL,SOC,SOCCUL,OVERALL
1,2,0.0,60.0,0.25,0.6667,0.3333,0.0,0.0,0.25,0.3333,...,0.5,0.75,0.3333,0.2,0.75,0.3182,0.2792,0.5,0.3896,0.3658


### PLots

In [ ]:
fcolor = '#ff0066'
mcolor = '#3333cc'
genderColor = fcolor if dfRow['GENDER'].iloc[0] == 1 else mcolor

‌

**PLot of economical status and sococultrual status**

The original demand of our proffsor was to create a 2D plot like this. This plot is more understandble comparing to 3D plot that you will see next.

In [ ]:
# TODO 1: scatterplot for ECO and SOCCUL
fig = px.scatter(ndf, x='ECO', y='SOC', color=df['GENDER'].copy().astype(str), size_max=10,
    color_discrete_map={'1': fcolor, '2': mcolor}, hover_data=['ID'])

# Customize the plot
fig.update_layout(title=dict(text='Dot Plot of Social Status', x=0.5, xanchor='center'),
    xaxis_title='Economical Score', yaxis_title='Cultural & Social Score',
    width=600, height=500,
    xaxis=dict(range=[0, 1]), yaxis=dict(range=[0, 1]),
    legend=dict(title='Gender'))

# Change the labels for the colors in the legend
for scatdata in fig.data:
    if scatdata.name == '1':
        scatdata.name = 'Female'
    else: scatdata.name = 'Male'


# TODO 2: Find the coresponding datapoint to ID + bold it
fig.add_trace(px.scatter(ndfRow, x='ECO', y='SOC', hover_data=['ID']).data[0]
    color_discrete_sequence=[genderColor], opacity=1, # color
    size=[10], size_max=10) #size

fig.show()

‌

**3D pLot of economical, social & cultrul status**


In [ ]:
#TODO 1: interactive 3d plot
fig = px.scatter_3d(ndf, x='ECO', y='CUL', z='SOC',
                    color=df['GENDER'].copy().astype(str),
                    color_discrete_map={'1': '#ff0066', '2': '#3333cc'},
                    size_max=10, hover_data=['ID'])

# Customize the plot
fig.update_layout(title=dict(text='Dot Plot of Social Status', x=0.5, xanchor='center'),
    scene = dict(xaxis_title='Economical Score', yaxis_title='Cultural Score', zaxis_title='Social Score',
                 xaxis=dict(range=[0,1]), yaxis=dict(range=[0,1]), zaxis=dict(range=[0,1])))

# Change the labels for the colors in the legend
for plot3d in fig.data:
    if plot3d.name == '1':
        plot3d.name = 'Female'
    else: plot3d.name = 'Male'


#TODO 2: Find the coresponding datapoint to ID + point to it with an arrow
fig.update_layout(scene=dict(annotations=[dict
    (x=ndfRow['ECO'].iloc[0], y=ndfRow['CUL'].iloc[0], z=ndfRow['SOC'].iloc[0],
    showarrow=True, arrowhead=2, arrowsize=1, arrowwidth=2, arrowcolor='black', #arrow
    # text and textbox detail
    text="Your Data Point",
    bgcolor="rgba(255, 255, 255, 0.8)", bordercolor="#c7c7c7", borderwidth=2, borderpad=4)]))

fig.show()

‌

**Overall comprasion**